In [ ]:
import pandas as pd

hr = pd.read_csv("data\heart_rate_7users_daily_avgHR_CLEAN_v2.csv")
steps = pd.read_csv("data\steps_7users_overtranied_full.csv")

hr['calendar_date'] = pd.to_datetime(hr['calendar_date'])
steps['calendar_date'] = pd.to_datetime(steps['calendar_date'])

data = pd.merge(hr, steps[['sdv_user_id', 'calendar_date', 'steps', 'steps_goal']], on=['sdv_user_id', 'calendar_date'], how ='inner')



